**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [初期設定](#toc3_)    
- [関数の定義](#toc4_)    
- [シミュレーションの実行](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは「2軸のロボットアーム」と「オブジェクト」間の衝突判定を行う方法を紹介します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0


pybullet build time: Nov 28 2023 23:45:17


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[初期設定](#toc0_)
床の生成、ボックスオブジェクトの生成、ロボットの生成、カメラ位置の設定などの初期設定を行います。

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# ロボットの読み込み
armStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple2d_arm.urdf",armStartPos, armStartOrientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
cameraDistance = 1.5
cameraYaw = 180.0 # deg
cameraPitch = -10 # deg
cameraTargetPosition = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# ボックスの読み込み
## ボックスの重さ、サイズ、位置·姿勢を決める
mass = 5 # kg
boxSize = [0.3, 0.4, 0.5]
position = [0.8, 0, 0.5]
orientation = [1, 0, 0, 0] # 四元数
boxCollisionId = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=boxSize, physicsClientId=physicsClient)
boxVisualId = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=boxSize, physicsClientId=physicsClient, rgbaColor=[1,0,0,1]) # 赤・半透明
boxBodyId = pybullet.createMultiBody(mass, boxCollisionId, boxVisualId, position, orientation, physicsClientId=physicsClient)

ven = Mesa
ven = Mesa
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc4_'></a>[関数の定義](#toc0_)
物体間の衝突の有効化/無効化に必要な、以下の関数を定義します。
- `GetNumLinks`：指定したオブジェクトのリンク数を取得
- `EnableCollisionBetweenObjects`：2つのオブジェクト間の衝突を有効/無効化する
- `JudgeContact`：2つのオブジェクトが接触しているかどうかを判定

In [3]:
def GetNumLinks(objId):
    """
    指定されたオブジェクトのリンク数を取得する。
    Args:
        obj (int): オブジェクトのインデックス。
        
    Returns:
        int: オブジェクトのリンク数。
    """
    currentLinkIdx = 0
    linkNum = 0
    while True:
        result = pybullet.getLinkState(objId, currentLinkIdx)
        if result is None:
            linkNum = currentLinkIdx + 1
            break
        currentLinkIdx += 1
    return linkNum

def EnableCollisionBetweenObjects(obj1Id, obj2Id, enable):
    """
    指定されたオブジェクト間の衝突を有効/無効にする。
    Args:
        obj1Idx (int): 1つ目ののオブジェクトのインデックス。
        obj2Idx (int): 2つ目ののオブジェクトのインデックス。
        enable (bool): 衝突を有効にする場合はTrue、無効にする場合はFalseを指定。
        
    Returns:
        None
    """
    # 各オブジェクトのリンク数を取得
    obj1LinkNum = GetNumLinks(obj1Id)
    obj2LinkNum = GetNumLinks(obj2Id)

    # 衝突判定の有効/無効を設定
    for obj1LinkIdx in range(-1, obj1LinkNum):
        for obj2LinkIdx in range(-1, obj2LinkNum):
            pybullet.setCollisionFilterPair(obj1Id, obj2Id, obj1LinkIdx, obj2LinkIdx, enable)

def JudgeCollision(obj1Id, obj2Id):
    """
    2つのオブジェクトが接触しているかどうかを判定する。
    Args:
        obj1Id (int): 1つ目のオブジェクトのインデックス。
        obj2Id (int): 2つ目のオブジェクトのインデックス。
        
    Returns:
        bool: 2つのオブジェクトが接触している場合はTrue、そうでない場合はFalse。
    """
    # 2つのオブジェクト間の接触情報を取得
    pts = pybullet.getClosestPoints(obj1Id, obj2Id, distance=100)

    # 2つのオブジェクトが接触しているかどうかを判定
    isCollision = False
    for pt in pts:
        distance = pt[8]
        # 2つのオブジェクト間で接触している点（=距離が0未満の点）があるかを判定
        if distance < 0.0:
            isCollision = True
            break
	
    return isCollision

# <a id='toc5_'></a>[シミュレーションの実行](#toc0_)
ロボットアームを動かし、オブジェクトとの衝突判定を行います。

下記コードを実行すると、GUI画面上のロボットアームをマウスで操作することができ、ロボットアームをボックスに触れさせると、画面上に「collision」と表示されます。

In [4]:
import time

pybullet.setRealTimeSimulation(1) # リアルタイムシミュレーションを有効化（GUI画面上で、オブジェクトをマウスで操作することが可能になる）

# ロボットアームとボックス間の衝突判定を無効化
EnableCollisionBetweenObjects(armId, boxBodyId, False)

# ロボットアームとボックスが衝突しているかどうかを画面上に表示
while (pybullet.isConnected()):

	# ロボットアームとボックスが衝突しているかどうかを判定
	isCollision = JudgeCollision(armId, boxBodyId)

	# 衝突している場合は画面上に表示
	if isCollision:
		pybullet.addUserDebugText("collision", [0.5, 0.0, 1.5], textSize=5, lifeTime=0.1, textColorRGB=[1,0,0])
	
	time.sleep(timeStep)